In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline

In [2]:
xlsx = pd.ExcelFile('depressed.xlsx')

C:\Users\User\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
data = pd.read_excel(xlsx, 'RapidMiner Data',header=0)
data

,Created-At,From-User,From-User-Id,To-User,To-User-Id,Language,Source,Text,Geo-Location-Latitude,Geo-Location-Longitude,Retweet-Count,Id
0,2021-10-04 21:02:35,MeidasTouch.com,1243560408025198593,NaN,-1,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",It's not just that Facebook undermined democra...,NaN,NaN,4872,1445026566690508802
1,2021-10-05 01:10:47,Rob Beckett,91798218,NaN,-1,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",I’m gonna be so depressed if I have no message...,NaN,NaN,791,1445089029511389188
2,2021-10-04 01:26:45,HaHa Davis,226340338,NaN,-1,en,"<a href=""http://twitter.com/download/iphone"" r...","I was 200 Pounds On The Left Lonely, Depressed...",NaN,NaN,2177,1444730658757152777
3,2021-10-05 10:03:56,Brandi✌️,572740851,NaN,-1,en,"<a href=""http://twitter.com/download/iphone"" r...",RT @MeidasTouch: It's not just that Facebook u...,NaN,NaN,4872,1445223199143182338
4,2021-10-05 10:03:48,Alex Da Great,1082086720122863621,NaN,-1,en,"<a href=""http://twitter.com/download/iphone"" r...",RT @Aaron_Skyguy: If anyone should be depresse...,NaN,NaN,31,1445223167262396417
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2021-10-03 18:09:04,Deepak Bhole,31098249,NaN,-1,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @ShefVaidya: Now #AryanKhan will soon do an...,NaN,NaN,1529,1444620513247055875
9996,2021-10-03 18:09:03,J da menace,3354104894,NaN,-1,en,"<a href=""http://twitter.com/download/iphone"" r...",u can be depressed n happy at the same time,NaN,NaN,1,1444620508549500937
9997,2021-10-03 18:09:03,TeaTreeJones????????,1222845303922933760,NaN,-1,en,"<a href=""http://twitter.com/download/iphone"" r...",RT @bangtanmum: @ronInBendigo I'm depressed at...,NaN,NaN,1,1444620508247441411
9998,2021-10-03 18:09:03,Arif Mamun,98069061,NaN,-1,en,"<a href=""http://twitter.com/download/iphone"" r...","RT @jasonhickel: We are a depressed, lonely an...",NaN,NaN,6182,1444620507047886850


## Exploratory Data Anaysis: EDA

In [4]:
# Drop unrelated columns
data = data.drop(['From-User', 'From-User-Id', 'Language', 'To-User', 'To-User-Id', 'Source', 'Geo-Location-Latitude', 'Geo-Location-Longitude','Retweet-Count'], axis=1)

In [5]:
data.set_index('Id', inplace = True)

In [6]:
data.head()

,Created-At,Text
Id,,
1445026566690508802,2021-10-04 21:02:35,It's not just that Facebook undermined democra...
1445089029511389188,2021-10-05 01:10:47,I’m gonna be so depressed if I have no message...
1444730658757152777,2021-10-04 01:26:45,"I was 200 Pounds On The Left Lonely, Depressed..."
1445223199143182338,2021-10-05 10:03:56,RT @MeidasTouch: It's not just that Facebook u...
1445223167262396417,2021-10-05 10:03:48,RT @Aaron_Skyguy: If anyone should be depresse...


In [7]:
print('Data size: ', data.shape)
print('Columns: ', data.columns)

Data size:  (10000, 2)
Columns:  Index(['Created-At', 'Text'], dtype='object')


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1445026566690508802 to 1444620493412347906
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Created-At  10000 non-null  datetime64[ns]
 1   Text        10000 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 234.4+ KB


## Data Preprocessing

In [9]:
# Find and drop duplicated data
data.Text.duplicated()

Id
1445026566690508802    False
1445089029511389188    False
1444730658757152777    False
1445223199143182338    False
1445223167262396417    False
                       ...  
1444620513247055875     True
1444620508549500937    False
1444620508247441411    False
1444620507047886850     True
1444620493412347906     True
Name: Text, Length: 10000, dtype: bool

In [10]:
data.Text.duplicated().sum()

5444

In [11]:
(~data.Text.duplicated()).sum()

4556

In [12]:
data.drop_duplicates(inplace=True, subset='Text', keep='last')

In [13]:
data

,Created-At,Text
Id,,
1445026566690508802,2021-10-04 21:02:35,It's not just that Facebook undermined democra...
1445089029511389188,2021-10-05 01:10:47,I’m gonna be so depressed if I have no message...
1444730658757152777,2021-10-04 01:26:45,"I was 200 Pounds On The Left Lonely, Depressed..."
1445223167262396417,2021-10-05 10:03:48,RT @Aaron_Skyguy: If anyone should be depresse...
1445223149595934721,2021-10-05 10:03:44,RT @hoshifan420: like the closest in color pal...
...,...,...
1444620555936833538,2021-10-03 18:09:14,"@ClaudetteGGibs1 When I lived up north, fall d..."
1444620513247055875,2021-10-03 18:09:04,RT @ShefVaidya: Now #AryanKhan will soon do an...
1444620508549500937,2021-10-03 18:09:03,u can be depressed n happy at the same time


#### Remove punctuations

In [14]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
def drop_punctuation(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [16]:
data['Text_punct'] = data['Text'].apply(lambda i: drop_punctuation(i))
data.head()

,Created-At,Text,Text_punct
Id,,,
1445026566690508802,2021-10-04 21:02:35,It's not just that Facebook undermined democra...,Its not just that Facebook undermined democrac...
1445089029511389188,2021-10-05 01:10:47,I’m gonna be so depressed if I have no message...,I’m gonna be so depressed if I have no message...
1444730658757152777,2021-10-04 01:26:45,"I was 200 Pounds On The Left Lonely, Depressed...",I was Pounds On The Left Lonely Depressed and...
1445223167262396417,2021-10-05 10:03:48,RT @Aaron_Skyguy: If anyone should be depresse...,RT AaronSkyguy If anyone should be depressed i...
1445223149595934721,2021-10-05 10:03:44,RT @hoshifan420: like the closest in color pal...,RT hoshifan like the closest in color palette ...


#### Tokenization - Converting a sentence into list of words

In [17]:
def token(text):
    text = re.split('\W+', text)
    return text

In [18]:
data['Text_token'] = data['Text_punct'].apply(lambda i: token(i.lower()))
data.head()

,Created-At,Text,Text_punct,Text_token
Id,,,,
1445026566690508802,2021-10-04 21:02:35,It's not just that Facebook undermined democra...,Its not just that Facebook undermined democrac...,"[its, not, just, that, facebook, undermined, d..."
1445089029511389188,2021-10-05 01:10:47,I’m gonna be so depressed if I have no message...,I’m gonna be so depressed if I have no message...,"[i, m, gonna, be, so, depressed, if, i, have, ..."
1444730658757152777,2021-10-04 01:26:45,"I was 200 Pounds On The Left Lonely, Depressed...",I was Pounds On The Left Lonely Depressed and...,"[i, was, pounds, on, the, left, lonely, depres..."
1445223167262396417,2021-10-05 10:03:48,RT @Aaron_Skyguy: If anyone should be depresse...,RT AaronSkyguy If anyone should be depressed i...,"[rt, aaronskyguy, if, anyone, should, be, depr..."
1445223149595934721,2021-10-05 10:03:44,RT @hoshifan420: like the closest in color pal...,RT hoshifan like the closest in color palette ...,"[rt, hoshifan, like, the, closest, in, color, ..."


#### Remove stopwords

In [19]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
stopword = nltk.corpus.stopwords.words('english')
stopword

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [22]:
data['Text_stopword'] = data['Text_token'].apply(lambda i: remove_stopwords(i))
data.head()

,Created-At,Text,Text_punct,Text_token,Text_stopword
Id,,,,,
1445026566690508802,2021-10-04 21:02:35,It's not just that Facebook undermined democra...,Its not just that Facebook undermined democrac...,"[its, not, just, that, facebook, undermined, d...","[facebook, undermined, democracies, home, abro..."
1445089029511389188,2021-10-05 01:10:47,I’m gonna be so depressed if I have no message...,I’m gonna be so depressed if I have no message...,"[i, m, gonna, be, so, depressed, if, i, have, ...","[gonna, depressed, messages, whatsapp, starts,..."
1444730658757152777,2021-10-04 01:26:45,"I was 200 Pounds On The Left Lonely, Depressed...",I was Pounds On The Left Lonely Depressed and...,"[i, was, pounds, on, the, left, lonely, depres...","[pounds, left, lonely, depressed, lost, months..."
1445223167262396417,2021-10-05 10:03:48,RT @Aaron_Skyguy: If anyone should be depresse...,RT AaronSkyguy If anyone should be depressed i...,"[rt, aaronskyguy, if, anyone, should, be, depr...","[rt, aaronskyguy, anyone, depressed, star, war..."
1445223149595934721,2021-10-05 10:03:44,RT @hoshifan420: like the closest in color pal...,RT hoshifan like the closest in color palette ...,"[rt, hoshifan, like, the, closest, in, color, ...","[rt, hoshifan, like, closest, color, palette, ..."


#### Lammatization/ Stemming - Tranforming any form of a word to its root word

In [23]:
poster = nltk.PorterStemmer()

In [24]:
def stemming(text):
    text = [poster.stem(word) for word in text]
    return text

In [25]:
data['Text_stemming'] = data['Text_stopword'].apply(lambda i: stemming(i))
data.head()

,Created-At,Text,Text_punct,Text_token,Text_stopword,Text_stemming
Id,,,,,,
1445026566690508802,2021-10-04 21:02:35,It's not just that Facebook undermined democra...,Its not just that Facebook undermined democrac...,"[its, not, just, that, facebook, undermined, d...","[facebook, undermined, democracies, home, abro...","[facebook, undermin, democraci, home, abroad, ..."
1445089029511389188,2021-10-05 01:10:47,I’m gonna be so depressed if I have no message...,I’m gonna be so depressed if I have no message...,"[i, m, gonna, be, so, depressed, if, i, have, ...","[gonna, depressed, messages, whatsapp, starts,...","[gonna, depress, messag, whatsapp, start, work]"
1444730658757152777,2021-10-04 01:26:45,"I was 200 Pounds On The Left Lonely, Depressed...",I was Pounds On The Left Lonely Depressed and...,"[i, was, pounds, on, the, left, lonely, depres...","[pounds, left, lonely, depressed, lost, months...","[pound, left, lone, depress, lost, month, late..."
1445223167262396417,2021-10-05 10:03:48,RT @Aaron_Skyguy: If anyone should be depresse...,RT AaronSkyguy If anyone should be depressed i...,"[rt, aaronskyguy, if, anyone, should, be, depr...","[rt, aaronskyguy, anyone, depressed, star, war...","[rt, aaronskyguy, anyon, depress, star, war, c..."
1445223149595934721,2021-10-05 10:03:44,RT @hoshifan420: like the closest in color pal...,RT hoshifan like the closest in color palette ...,"[rt, hoshifan, like, the, closest, in, color, ...","[rt, hoshifan, like, closest, color, palette, ...","[rt, hoshifan, like, closest, color, palett, j..."


In [26]:
wordnet = nltk.WordNetLemmatizer()

In [27]:
def lemmatizer(text):
    text = [wordnet.lemmatize(word) for word in text]
    return text

In [28]:
data['Text_lammetization'] = data['Text_stopword'].apply(lambda i: lemmatizer(i))
data.head()

,Created-At,Text,Text_punct,Text_token,Text_stopword,Text_stemming,Text_lammetization
Id,,,,,,,
1445026566690508802,2021-10-04 21:02:35,It's not just that Facebook undermined democra...,Its not just that Facebook undermined democrac...,"[its, not, just, that, facebook, undermined, d...","[facebook, undermined, democracies, home, abro...","[facebook, undermin, democraci, home, abroad, ...","[facebook, undermined, democracy, home, abroad..."
1445089029511389188,2021-10-05 01:10:47,I’m gonna be so depressed if I have no message...,I’m gonna be so depressed if I have no message...,"[i, m, gonna, be, so, depressed, if, i, have, ...","[gonna, depressed, messages, whatsapp, starts,...","[gonna, depress, messag, whatsapp, start, work]","[gonna, depressed, message, whatsapp, start, w..."
1444730658757152777,2021-10-04 01:26:45,"I was 200 Pounds On The Left Lonely, Depressed...",I was Pounds On The Left Lonely Depressed and...,"[i, was, pounds, on, the, left, lonely, depres...","[pounds, left, lonely, depressed, lost, months...","[pound, left, lone, depress, lost, month, late...","[pound, left, lonely, depressed, lost, month, ..."
1445223167262396417,2021-10-05 10:03:48,RT @Aaron_Skyguy: If anyone should be depresse...,RT AaronSkyguy If anyone should be depressed i...,"[rt, aaronskyguy, if, anyone, should, be, depr...","[rt, aaronskyguy, anyone, depressed, star, war...","[rt, aaronskyguy, anyon, depress, star, war, c...","[rt, aaronskyguy, anyone, depressed, star, war..."
1445223149595934721,2021-10-05 10:03:44,RT @hoshifan420: like the closest in color pal...,RT hoshifan like the closest in color palette ...,"[rt, hoshifan, like, the, closest, in, color, ...","[rt, hoshifan, like, closest, color, palette, ...","[rt, hoshifan, like, closest, color, palett, j...","[rt, hoshifan, like, closest, color, palette, ..."


In [29]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text = re.sub('[0-9]+', '', text)
    text = re.split('\W+', text)    # tokenization
    text = [poster.stem(word) for word in text if word not in stopword]  # remove stopwords and stemming
    text = [wordnet.lemmatize(word) for word in text] #lemmatizer
    return text

In [30]:
countVectorizer = CountVectorizer(analyzer=clean_text) 
countVector = countVectorizer.fit_transform(data['Text'])
print('{} Number of Text has {} words'.format(countVector.shape[0], countVector.shape[1]))
#print(countVectorizer.get_feature_names())

4556 Number of Text has 10598 words


In [31]:
# data.to_csv('data_test.csv')

#### Wordcloud Visualization 
ต้องสร้าง target column ก่อน

In [32]:
#conda install -c conda-forge wordcloud=1.6.0 

In [33]:
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator